<a href="https://colab.research.google.com/github/jerry-mouse-2333/colab_save/blob/master/more_sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ***step1. 导入必要的包***

In [1]:
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os
import sys
import h5py
import tensorflow.compat.v1 as tf
tf.compat.v1.disable_eager_execution()
from functools import partial
from tensorflow.compat.v1.keras.initializers import glorot_uniform

tf.reset_default_graph()
print('ready')

ready


## ***step2. 防 jupyter dead kernel***

In [2]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True

session = InteractiveSession(config=config)
session.close()

# InteractiveSession.close(session)
print('ready')
## ***step3. make def for plot***
#### **step3.1. basic 画布和标签**
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

#定义子图、x轴、y轴的标签大小

(plt.rcParams['axes.labelsize'],plt.rcParams['xtick.labelsize'],plt.rcParams['ytick.labelsize'])  = (14,12,12)

#定义图像显示的dpi、保存的dpi，显示的窗口大小

(plt.rcParams['figure.dpi'],plt.rcParams['savefig.dpi'],plt.rcParams['figure.figsize'])=(300,300,(80,60))

print('ready')
###### **step3.1.1. plot** (desired_shape:希望展示的图片大小)
def plot_image(image, desired_shape=[128, 128]):
    
    plt.imshow(image.reshape(desired_shape), cmap="Greys", interpolation="nearest")
    plt.axis("off")
print('ready')
#### **step3.2. where to save**
# Where to save the figures

root_dir = "/content/drive/My Drive/3result of opts"
project = "more_samples"
path = []
(img_save_path,summary_save_path,ckpt_save_path) = (os.path.join(root_dir,project,"images"),os.path.join(root_dir,project,"events"),os.path.join(root_dir,project,"models"))
# summary_save_path = os.path.join(root_dir,sub_dir,"events")
# ckpt_save_path = os.path.join(root_dir,sub_dir,"models")
path.append(img_save_path)
path.append(summary_save_path)
path.append(ckpt_save_path)
for i in range(len(path)):
  os.makedirs(path[i],exist_ok=True)
print(path)

#tight_layout: 使子图布满图像区域
def save_fig(fig_id, tight_layout=True):
    save_name = os.path.join(img_path, fig_id + ".png")
    print("Saving figure", fig_id)
    # os.makedirs(path, exist_ok=True)
    
#    if not os.path.exists(os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)):

#        os.makedirs(os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID))
    if tight_layout:
        plt.tight_layout()
    plt.savefig(save_name, format='png', dpi=300)
print('ready')

ready
ready
ready
['/content/drive/My Drive/3result of opts/more_samples/images', '/content/drive/My Drive/3result of opts/more_samples/events', '/content/drive/My Drive/3result of opts/more_samples/models']
ready


## **step4. 散度**

In [3]:
def kl_divergence(p, q):
    # Kullback Leibler divergence
    return p * tf.log(p / q) + (1 - p) * tf.log((1 - p) / (1 - q))
print('ready')
## **step4. 网络结构**
#### **step4.1. 参数设置**
# filters=[32,16,32]
filters=[64,32,16]
conv_kernel=3

batch_size = 64
ephochs = 20

#原始图片尺寸
width = 128
height = 128
depth = 1

#输入层
input_dim = (128,128)

#编码层输出
en_output=16

#解码层输出
output_dim = input_dim

#学习率与正则参数
learning_rate = 0.01
l2_reg = 0.0001

#散度参数
sparsity_target = 0.1
sparsity_weight = 0.2

#train_method2：全连接层
en1 = 300 #hid_1 of encoder
en2 = 150  
de1 = en2 #hid_1 of decoder
de2 = en1



print('ready')

ready
ready


无奈之举，想设置kernel_size=5，strides=3，但无法使得输入输出一致

#### **step4.2. 网络结构**
###### **step4.21. encoder**

In [4]:
#收集变量绘制其histogram等
tensor_list=[]
stds=3


def kernel_size(na,input_channel,output_channel):
  width = 5
  height = 5
  input_channel = input_channel
  output_channel = output_channel
  sp = [width,height,input_channel,output_channel]
  k=tf.get_variable(name=na,shape=sp,dtype=tf.float32,initializer=glorot_uniform(),trainable=True)
  print(k)
  # k = tf.Variable(trainable=True,shape=shape,glorot_uniform(), dtype=tf.float32)


  print("kernel_shape:",sp)
  return k
# def output_size(inputs,chls,batch_size=batch_size,k=5,stds=stds):
  
#   width = (inputs.shape[1]-k+2)//stds+1
#   hieght = width
#   channels = chls
#   return [batch_size,width,height,channels]

out_shape=[]


with tf.name_scope("encoder") as scope:

    def en_module(en_inputs,en_output_dim,filters,k=conv_kernel):
        x=en_inputs
        
        out_shape.append([64,128,128,64])
        i=0
        for _ in filters:
            print(_)
            # x = tf.nn.conv2d(x,kernel_size(_,x.shape[-1]),output_size(x,_),stds,'conv_'+str(i))
            #conv2d 中，没有output_shape,输入的参数为：input，kernel[4D],strides,padding)
            # print('input_dim:',x.shape[-1])
            x = tf.nn.conv2d(x,kernel_size('en_k_'+str(i),x.shape[-1],_),stds,[[0,0],[1,1],[1,1],[0,0]],name='conv_'+str(i))
            tensor_list.append(x)
            out_shape.append(x.shape)
            # print('encoder_layer_shape',x.shape)
            tf.add_to_collection('reuse', x)
            x=tf.nn.leaky_relu(x, alpha=0.01,name='act_'+str(i))
            tensor_list.append(x)
            tf.add_to_collection('reuse', x)
            x = tf.layers.batch_normalization(x,axis=-1,name='batch_norm_'+str(i))
            tensor_list.append(x)
            
            tf.add_to_collection('reuse', x)
            i+=1
        
        shape=x.get_shape().as_list()

        print('out_shape:',len(out_shape))
        flat1 = tf.layers.flatten(x,name='flattened')
        #print(flat1.shape)

        #必须使用sigmoid激活使得KL_loss!=Nan
        half_output = my_dense_layer(flat1,en_output_dim,activation = "sigmoid",name="encoder_output")

        tf.add_to_collection('reuse', half_output)

        return shape,flat1,half_output

###### **step4.21. decoder**
with tf.name_scope("decoder") as scope:
  def de_module(de_inputs,depth,filters,k=conv_kernel):
    x = de_inputs
    i=0
    def o_shape(a,output_dim,bs=batch_size):
      dim=output_dim
      width=a[1]

      return([bs,width,width,dim])
    f=[32,64]
    # for _ in filters[::-1]:
    for _ in f:
    # kernel_size函数中，由于transpose是out,in，所以此处反着输入
        # if(i==1):
        #   x = tf.nn.conv2d_transpose(x,kernel_size(_,x.shape[-1]),out_shape[len(out_shape)-2-i],padding=[0,1,1,1,0,0],name='conv_tran_'+str(i))
        print(i)
        print('本次输入：',x.shape)
        print("本次目标：",out_shape[len(out_shape)-2-i])
                
        idx = len(out_shape)-2-i
        if (i==0):
          x = tf.nn.conv2d_transpose(x,kernel_size('de_k_'+str(i),_,x.shape[-1]),o_shape(out_shape[idx],_),3,'VALID',name='conv_tran_'+str(i))
        else:
          x = tf.nn.conv2d_transpose(x,kernel_size('de_k_'+str(i),_,x.shape[-1]),o_shape(out_shape[idx],_),3,name='conv_tran_'+str(i))
        print(x.shape==out_shape[len(out_shape)-2-i])
        x=tf.nn.leaky_relu(x, alpha=0.01,name='de_act_'+str(i))
        x = tf.layers.batch_normalization(x,axis=-1,name='de_batch_norm_'+str(i))
        i+=1   
      
      #整合所有通道
    print('给output的输入：',x.shape)
#由于默认的stride是1，一开始报错输入的input与期待的不同，分别为42，128
    outputs_op = tf.nn.conv2d_transpose(x,kernel_size('output_k',depth,x.shape[-1]),[64,128,128,1],3,'VALID',name='outputs_op')
    # ,strides=3,[64,128,128,1],
    # outputs_op = tf.layers.conv2d(x,depth,1,1,padding='same',activation="sigmoid",name='outputs_op')
    # print('output.shape:',outputs_op.shape)
      #观察最后的重建效果
    tf.add_to_collection('reuse', outputs_op)
    outputs=outputs_op
    return outputs


###### **step4.2.3. 正式布置graph**
# tf.reset_default_graph()

inputs = tf.placeholder(tf.float32, shape=[batch_size,input_dim[0],input_dim[1],depth])#?,128,128
print(inputs)
#把input加入到将来要用的集合中
tf.add_to_collection('reuse', inputs)
print(inputs.shape)

#he_init = tf.contrib.layers.variance_scaling_initializer() # He initialization
he_init = tf.variance_scaling_initializer()
l2_regularizer = tf.keras.regularizers.l2(l2_reg)
my_dense_layer = partial(tf.layers.dense,activation=tf.nn.elu,
                             kernel_initializer=he_init,
                             kernel_regularizer=l2_regularizer)
#en_inputs = tf.expand_dims(inputs,axis=-1,name="encoder_input")
en_inputs=inputs

shape,flat1,half_output = en_module(en_inputs,en_output_dim=en_output,filters=filters)
print('shape_in_encoder:',shape)
print(half_output.name)


decode_input = my_dense_layer(half_output,np.prod(flat1.shape[1:]),name="decoder_input")#现为（？，32*32*8）
#print(decode_input)
decode_input_reshape= tf.reshape(decode_input,shape=(-1,shape[1],shape[2],shape[3]))#how to reshape it to(?,32,32,8)
print('decode_input.shape:',decode_input_reshape)
outputs = de_module(decode_input_reshape,depth=depth,filters=filters)
print(outputs)
print(outputs.name)

encode_mean = tf.reduce_mean(half_output, axis=0)

## step4 可视化设置
with tf.name_scope("loss") as scope:
    kl=kl_divergence(sparsity_target, encode_mean)

    sparsity_loss = tf.reduce_sum(kl_divergence(sparsity_target, encode_mean))
    reconstruction_loss = tf.reduce_mean(tf.square(outputs - en_inputs))
    loss = reconstruction_loss + sparsity_weight * sparsity_loss
    #tensor_list.append(loss)
optimizer = tf.train.AdamOptimizer(learning_rate)
training_op = optimizer.minimize(loss)

tf.summary.scalar("loss",loss)
# 用来显示标量信息，常用于画出loss、accuracy曲线
print(tensor_list)
for i in range(len(tensor_list)):
    name=tensor_list[i]
    #print(name.dtype)
    tf.summary.histogram(str(name),name) 
# 用来收集tensor，可显示出训练过程中变量的分布情况


init = tf.global_variables_initializer()
saver = tf.train.Saver() # not shown in the book
print('ok')

Tensor("Placeholder:0", shape=(64, 128, 128, 1), dtype=float32)
(64, 128, 128, 1)
64
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
<tf.Variable 'en_k_0:0' shape=(5, 5, 1, 64) dtype=float32>
kernel_shape: [5, 5, 1, 64]
Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.BatchNormalization` documentation).
Instructions for updating:
Please use `layer.__call__` method instead.
32
<tf.Variable 'en_k_1:0' shape=(5, 5, 64, 32) dtype=float32>
kernel_shape: [5, 5, 64, 32]
16
<tf.Variable 'en_k_2:0' shape=(5, 5, 32, 16) dtype=float32>
kernel_shape: [5, 5, 32, 16]
out_shape: 4
Instructions for updating:
Use keras.layers.Flatten instead.
Instructions for updating:
Use keras.layers.Dense instead.
shape_in_encoder: [64, 4, 4, 16]
encoder_output/Sigmoid:0
decode_input.shape: Tensor("Reshape:0", shape=(64, 4, 4, 16), dtype=float32)


In [5]:
os.makedirs(summary_save_path, exist_ok=True)
print(summary_save_path)
writer= tf.summary.FileWriter(summary_save_path,tf.get_default_graph())
#获取当前默认的保存图，并在tensorboard graph中展示
merge_op = tf.summary.merge_all()
#执行所有summary操作
print("ready")

/content/drive/My Drive/3result of opts/more_samples/events
ready


## **step5. 开始训练与结果保存**

In [6]:
from tensorflow.python.client import device_lib

# 列出所有的本地机器设备
local_device_protos = device_lib.list_local_devices()
# 打印
#     print(local_device_protos)

# 只打印GPU设备
[print(x) for x in local_device_protos if x.device_type == 'GPU']


name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 15701401920
locality {
  bus_id: 1
  links {
  }
}
incarnation: 3201175561766791674
physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"



[None]

In [0]:
data_path = '/content/drive/Shared drives/shared/kinds of datasets/dataset.h5'
h5_file=h5py.File(data_path,'r')


In [8]:
X_train=np.array(h5_file['X_train'])
#X_test=np.array(h5_file['X_test'])
print(X_train.shape)

(14316, 128, 128)


In [9]:
print(batch_size)
def get_batch(dataset,ephcos,batch_size):

    input_queue = tf.train.slice_input_producer([dataset],num_epochs=ephochs,shuffle=True)

    X_batch = tf.train.batch(input_queue, batch_size=batch_size, num_threads=1, capacity=64)
    print(X_batch.shape)
    return X_batch
X_batch= get_batch(X_train,ephochs,batch_size)

64
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(tuple(tensor_list)).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.range(limit).shuffle(limit).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensors(tensor).repeat(num_epochs)`.
Instructions for updating:
Prefer Dataset.range instead.
Instructions for updating:
To construct input pipelin

In [10]:
ckpt_name = os.path.join(ckpt_save_path,"nnconv2d_batch64_allsamples.ckpt")
print(ckpt_name)

/content/drive/My Drive/3result of opts/more_samples/models/nnconv2d_batch64_allsamples.ckpt


In [0]:
with tf.Session() as sess:
    init.run()
    sess.run(tf.local_variables_initializer())
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess, coord)
    try:
        for epoch in range(ephochs):
            n_batches = X_train.shape[0] // batch_size

            print(n_batches)
            for iteration in range(n_batches):
                # print("\r{}%".format(100 * iteration // n_batches), end="") # not shown in the book
                sys.stdout.flush()
                x_batch=sess.run(X_batch)
                
                x_batch = np.expand_dims(x_batch,axis=-1)
                # print(x_batch.shape)
                sess.run(training_op, feed_dict={inputs: x_batch})
                result=sess.run(merge_op, feed_dict={inputs: x_batch})
            writer.add_summary(result,epoch)
                
            reconstruction_loss_val, sparsity_loss_val, loss_val = sess.run([reconstruction_loss,sparsity_loss, loss], feed_dict={inputs: x_batch})
            #print(sess.run(kl,feed_dict={X: x_batch}))
            print("\r{}".format(epoch), "Train MSE:", reconstruction_loss_val, "\tSparsity loss:",sparsity_loss_val, "\tTotal loss:", loss_val)           # not shown
              
    except tf.errors.OutOfRangeError:
        print("done")
    finally:
        coord.request_stop()
    coord.join(threads)
    saver.save(sess,ckpt_name) 
    print('ok')


Instructions for updating:
To construct input pipelines, use the `tf.data` module.
223
0 Train MSE: 0.04337091 	Sparsity loss: 0.018707246 	Total loss: 0.04711236
223
1 Train MSE: 0.0327906 	Sparsity loss: 0.0030129403 	Total loss: 0.03339319
223
2 Train MSE: 0.026954042 	Sparsity loss: 0.002675255 	Total loss: 0.027489092
223
3 Train MSE: 0.023942614 	Sparsity loss: 0.0012972972 	Total loss: 0.024202073
223
4 Train MSE: 0.021967836 	Sparsity loss: 0.002294641 	Total loss: 0.022426764
223
5 Train MSE: 0.020657826 	Sparsity loss: 0.004872345 	Total loss: 0.021632295
223
6 Train MSE: 0.020581625 	Sparsity loss: 0.0021744273 	Total loss: 0.02101651
223
7 Train MSE: 0.018996457 	Sparsity loss: 0.004293481 	Total loss: 0.019855153
223
8 Train MSE: 0.017882545 	Sparsity loss: 0.0014921343 	Total loss: 0.018180972
223
9 Train MSE: 0.018945768 	Sparsity loss: 0.0032864003 	Total loss: 0.01960305
223
10 Train MSE: 0.017273005 	Sparsity loss: 0.0021608227 	Total loss: 0.01770517
223
11 Train MSE

## 1st try : 
- 1. raw_dataset(float32,255,9000张，8个G)
-   X_batch= get_batch(X_train[0:1000],ephochs,batch_size) 报错：tensor不能大于2G
-   ***solution:*** 只使用1000张X_train
- 2. train 过程中OOM

## 2nd try:
- to solve OOM: batch_size=32
- 可
- but **Nan** in summary histogram for: Tensor__conv_0/BiasAdd_0___shape__None__240__320__32___dtype_float32_
-     try: 1. 更改学习率0.01->0.001(useless)
        2. 更改loss为交叉熵


## ***3rd: Most exciting!!!***
- dataset：灰度化的10，000张
- 但是仅用1000张测试，否则OOM
- 灰度化之后，不再报 ***Nan*** 的错了！！！